In [111]:
from config import *
from utils import *
N = 1

In [112]:
0x6162 == 0b0110000101100010
# convert str(63) to 0x63
def str2hex(s):
    return int(s, 16)
str2hex('61')==0x61
def str2bin(s):
    return int(s, 2)
str2bin('01100001')==0x61

True

# Add Padding

In [113]:
message = "abc"
print("Message:", message)
print("Message(hex):", convert_to_hex(message))
print("Message(bin):", convert_to_bin(message))
print("------------------------------------------------------------")
padding_message = add_padding(message, N)
print("Padding message (hex):", padding_message)
print(f"Padding message length: {len(padding_message)} hex")
print("------------------------------------------------------------")
padding_message_bin = add_padding(message, N, 'bin')
print("Padding message (bin):", padding_message_bin)
print(f"Padding message length: {len(padding_message_bin)} bits")
print("------------------------------------------------------------")
padding_message_int_value = str2hex(padding_message)
print("Integer value:", padding_message_int_value)


Message: abc
Message(hex): 616263
Message(bin): 011000010110001001100011
------------------------------------------------------------
Padding message (hex): 6162638000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
Padding message length: 256 hex
------------------------------------------------------------
Padding message (bin): 01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

# Initial Hash

In [114]:
mp.dps = 50
s.sieve[1] 
first_eight_primes = s.sieve[1:9] 
initial_hash_values = [hex2int(sqrt_64bits_hex(i)) for i in first_eight_primes]
print("Intenger values initial hash",initial_hash_values)
print("------------------------------------------------------------")
print("Initial hash (hex):", [(sqrt_64bits_hex(i)) for i in first_eight_primes])
keys = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
dict_hash = dict(zip(keys, initial_hash_values))

Intenger values initial hash [7640891576956012808, 13503953896175478587, 4354685564936845355, 11912009170470909681, 5840696475078001361, 11170449401992604703, 2270897969802886507, 6620516959819538809]
------------------------------------------------------------
Initial hash (hex): ['6a09e667f3bcc908', 'bb67ae8584caa73b', '3c6ef372fe94f82b', 'a54ff53a5f1d36f1', '510e527fade682d1', '9b05688c2b3e6c1f', '1f83d9abfb41bd6b', '5be0cd19137e2179']


In [115]:
initial_hash = {
    'a': 0x6a09e667f3bcc908,
    'b': 0xbb67ae8584caa73b,
    'c': 0x3c6ef372fe94f82b,
    'd': 0xa54ff53a5f1d36f1,
    'e': 0x510e527fade682d1,
    'f': 0x9b05688c2b3e6c1f,
    'g': 0x1f83d9abfb41bd6b,
    'h': 0x5be0cd19137e2179,
}
dict_hash == initial_hash

True

# Process Padding Message

In [116]:
def ROTR(n, bits):
    return ((n >> bits) | (n << (64 - bits))) & 0xFFFFFFFFFFFFFFFF

def Sigma0(W, option= "Word"):
    if option == "Byte":
        return ROTR(W, 28) ^ ROTR(W, 34) ^ ROTR(W, 39)
    elif option == "Word":
        return ROTR(W, 1) ^ ROTR(W, 8) ^ (W >> 7)

def Sigma1(W, option= "Word"):
    if option == "Byte":
        return ROTR(W, 14) ^ ROTR(W, 18) ^ ROTR(W, 41)
    elif option == "Word":
        return ROTR(W, 19) ^ ROTR(W, 61) ^ (W >> 6)

In [117]:
def split_message(padding_message_bin):
    Word = [int(padding_message_bin[i:i+64], 2) for i in range(0, len(padding_message_bin), 64)]
    for i in range(16, 80):
        Word.append((Sigma1(Word[i-2], option= "Word") + Word[i-7] + Sigma0(Word[i-15], option= "Word") + Word[i-16]) & 0xFFFFFFFFFFFFFFFF)
    return Word

def split_message_hex(padding_message):
    Word = [int(padding_message[i:i+16], 16) for i in range(0, len(padding_message), 16)]
    for i in range(16, 80):
        Word.append((Sigma1(Word[i-2], option= "Word") + Word[i-7] + Sigma0(Word[i-15], option= "Word") + Word[i-16]) & 0xFFFFFFFFFFFFFFFF)
    return Word

print(split_message(padding_message_bin) == split_message_hex(padding_message))
Words = split_message(padding_message_bin)
print("List Words:", Words)
print(len(Words))
print(f"Word (hex values): {[hex(i)[2:] for i in Words]}")
print("------------------------------------------------------------")
print(f"Word (bit values): {[bin64bits(i) for i in [hex(i)[2:] for i in Words]]}")

True
List Words: [7017280570803617792, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 7017280570803617792, 844424930132160, 762966109232037891, 13195750147587, 6097581589540473238, 54043414571793408, 1483655128544091580, 7126213039481127152, 8602118586805487511, 17389888485336023043, 14567336803169050610, 6777467420161596722, 16310457152051956578, 11476377395010010066, 12906063273572982575, 1051720670150476534, 11866753008450386212, 12477396688234164614, 4356201549416458046, 7840148015528850751, 14121359558264892382, 6614793513367853908, 4522218185451149906, 2892552721718560235, 352189473509599370, 9030584440117125351, 12607724378241962598, 829913954127880271, 12787094207806812211, 6729427763312036062, 5477573448673053883, 16621003149437336899, 16999664878006829468, 3899695832832618899, 3357496273253151093, 3682082552034448227, 11294004811417749973, 2836744397419860151, 4287662693712260952, 938603131418550565, 8445351822503640832, 13301424169519149285, 2637789678536900255, 5398300769020

In [118]:
def Ch(e,f,g):
    return (e & f) ^ (~e & g)

def Maj(a,b,c):
    return (a & b) ^ (a & c) ^ (b & c)

In [119]:
def SHA512(message,initial_hash):
    padding_message = add_padding(message, N, 'bin')
    blocks = [padding_message[i:i+1024] for i in range(0, len(padding_message), 1024)]
    for block in (blocks):
        Words = split_message(block)
        a,b,c,d,e,f,g,h = initial_hash['a'],initial_hash['b'],initial_hash['c'],initial_hash['d'],initial_hash['e'],initial_hash['f'],initial_hash['g'],initial_hash['h']
        for i in range(len(Words)):
            T1 = (h + Ch(e,f,g) + Sigma1(e, option="Byte") + Words[i] + Key[i])%2**64
            T2 = (Sigma0(a, option="Byte") + Maj(a, b, c))%2**64
            h = g
            g = f
            f = e
            e = (d + T1) & 0xFFFFFFFFFFFFFFFF
            d = c
            c = b
            b = a
            a = (T1 + T2) & 0xFFFFFFFFFFFFFFFF
        initial_hash['a'] = (a + initial_hash['a']) & 0xFFFFFFFFFFFFFFFF
        initial_hash['b'] = (b + initial_hash['b']) & 0xFFFFFFFFFFFFFFFF
        initial_hash['c'] = (c + initial_hash['c']) & 0xFFFFFFFFFFFFFFFF
        initial_hash['d'] = (d + initial_hash['d']) & 0xFFFFFFFFFFFFFFFF
        initial_hash['e'] = (e + initial_hash['e']) & 0xFFFFFFFFFFFFFFFF
        initial_hash['f'] = (f + initial_hash['f']) & 0xFFFFFFFFFFFFFFFF
        initial_hash['g'] = (g + initial_hash['g']) & 0xFFFFFFFFFFFFFFFF
        initial_hash['h'] = (h + initial_hash['h']) & 0xFFFFFFFFFFFFFFFF
    return initial_hash


In [120]:
plain_text = "abc"
print("Plain text:", plain_text)
cipher_text = SHA512(plain_text, initial_hash)
print("Last Hash:", cipher_text)

Plain text: abc
Last Hash: {'a': 15974045371385084602, 'b': 14718171817514647857, 'c': 1362051152550133410, 'd': 765311659573367706, 'e': 2419164356178592168, 'f': 3943530547489205181, 'g': 4993722480620005390, 'h': 3069987439919277215}


In [121]:
# Concat all hash values
print("SHA-512:", "".join([hex(i)[2:] for i in cipher_text.values()]))

SHA-512: ddaf35a193617abacc417349ae20413112e6fa4e89a97ea2a9eeee64b55d39a2192992a274fc1a836ba3c23a3feebbd454d4423643ce80e2a9ac94fa54ca49f


In [1]:
import hashlib

text = 'abc'

m = hashlib.sha512(text.encode('UTF-8'))
print(m.hexdigest())

ddaf35a193617abacc417349ae20413112e6fa4e89a97ea20a9eeee64b55d39a2192992a274fc1a836ba3c23a3feebbd454d4423643ce80e2a9ac94fa54ca49f
